In [1]:
import numpy as np
import pandas as pd
import warnings
import os
from tqdm import tqdm
from sklearn import preprocessing, metrics
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

In [2]:
df_history_action = pd.read_pickle('./temp/action_history.plk')
df_feature = pd.read_pickle('./temp/base_feature.plk')
df_courier = pd.read_pickle('./temp/courier.plk')
df_order = pd.read_pickle('./temp/order.plk')
df_distance = pd.read_pickle('./temp/distance.plk')



In [3]:
seed = 2020
df_feature.head()

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id
0,10007871,0,2100074550065333539,121.630997,39.142343,DELIVERY,1580528963,20200201,train,1.0,20200201100078710,0
1,10007871,0,2100074550779577850,121.630997,39.142343,PICKUP,1580529129,20200201,train,0.0,20200201100078710,1
2,10007871,0,2100074550779577850,121.630997,39.142343,DELIVERY,1580529444,20200201,train,0.0,20200201100078710,2
0,10007871,1,2100074555638285402,121.631208,39.142519,PICKUP,1580532225,20200201,train,1.0,20200201100078711,3
1,10007871,1,2100074554118800474,121.631208,39.142519,PICKUP,1580532227,20200201,train,0.0,20200201100078711,4


# 历史订单信息

In [5]:
df_history_action.head()

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,group
120,10007871,0,2100074548854622111,121.630997,39.142343,PICKUP,1580527779,20200201,train,20200201100078710
121,10007871,0,2100074548854622111,121.630997,39.142343,DELIVERY,1580528077,20200201,train,20200201100078710
122,10007871,0,2100074550065333539,121.630997,39.142343,PICKUP,1580528622,20200201,train,20200201100078710
126,10007871,1,2100074553896437081,121.631208,39.142519,PICKUP,1580530391,20200201,train,20200201100078711
127,10007871,1,2100074553896437081,121.631208,39.142519,DELIVERY,1580531150,20200201,train,20200201100078711


In [6]:
#df_feature.head()

In [7]:
#df_courier.head()

In [8]:
#df_order.head()

In [9]:
df_distance.head()

,courier_id,wave_index,tracking_id,source_type,source_lng,source_lat,target_tracking_id,target_type,target_lng,target_lat,grid_distance,date,group
0,100002543,0,2100074536706869066,ASSIGN,121.641337,39.141340,2100074542836254538,ASSIGN,121.635755,39.141497,542.0,20200201,202002011000025430
1,100002543,0,2100074542836254538,PICKUP,121.647214,39.135549,2100074542836254538,ASSIGN,121.635755,39.141497,2042.0,20200201,202002011000025430
2,100002543,0,2100074542836254538,DELIVERY,121.647324,39.126481,2100074542836254538,ASSIGN,121.635755,39.141497,3343.0,20200201,202002011000025430
3,100002543,0,2100074536706869066,PICKUP,121.647214,39.135549,2100074542836254538,ASSIGN,121.635755,39.141497,2042.0,20200201,202002011000025430
4,100002543,0,2100074536706869066,DELIVERY,121.655694,39.123121,2100074542836254538,ASSIGN,121.635755,39.141497,3945.0,20200201,202002011000025430


In [10]:
# 获取 wave 最后一次 step 信息
df_temp = df_history_action.groupby(['group'])['expect_time'].apply(
    lambda x: x.values.tolist()[-1]).reset_index()
df_temp.columns = ['group', 'current_time']
df_feature = df_feature.merge(df_temp, how='left')

df_temp = df_history_action.groupby(['group'])['tracking_id'].apply(
    lambda x: x.values.tolist()[-1]).reset_index()
df_temp.columns = ['group', 'last_tracking_id']
df_feature = df_feature.merge(df_temp, how='left')

df_temp = df_history_action.groupby(['group'])['action_type'].apply(
    lambda x: x.values.tolist()[-1]).reset_index()
df_temp.columns = ['group', 'last_action_type']
df_feature = df_feature.merge(df_temp, how='left')

# distance 表相关特征

In [11]:
df_distance.head()

,courier_id,wave_index,tracking_id,source_type,source_lng,source_lat,target_tracking_id,target_type,target_lng,target_lat,grid_distance,date,group
0,100002543,0,2100074536706869066,ASSIGN,121.641337,39.141340,2100074542836254538,ASSIGN,121.635755,39.141497,542.0,20200201,202002011000025430
1,100002543,0,2100074542836254538,PICKUP,121.647214,39.135549,2100074542836254538,ASSIGN,121.635755,39.141497,2042.0,20200201,202002011000025430
2,100002543,0,2100074542836254538,DELIVERY,121.647324,39.126481,2100074542836254538,ASSIGN,121.635755,39.141497,3343.0,20200201,202002011000025430
3,100002543,0,2100074536706869066,PICKUP,121.647214,39.135549,2100074542836254538,ASSIGN,121.635755,39.141497,2042.0,20200201,202002011000025430
4,100002543,0,2100074536706869066,DELIVERY,121.655694,39.123121,2100074542836254538,ASSIGN,121.635755,39.141497,3945.0,20200201,202002011000025430


In [12]:
df_distance = df_distance.rename(columns={'tracking_id': 'last_tracking_id',
                                          'source_type': 'last_action_type', 'target_tracking_id': 'tracking_id', 'target_type': 'action_type'})
df_feature = df_feature.merge(df_distance.drop(
    ['courier_id', 'wave_index', 'date'], axis=1), how='left')

In [13]:
df_feature.head()

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance
0,10007871,0,2100074550065333539,121.630997,39.142343,DELIVERY,1580528963,20200201,train,1.0,20200201100078710,0,1580528622,2100074550065333539,PICKUP,121.631219,39.141811,121.632084,39.146201,707.0
1,10007871,0,2100074550779577850,121.630997,39.142343,PICKUP,1580529129,20200201,train,0.0,20200201100078710,1,1580528622,2100074550065333539,PICKUP,121.631219,39.141811,121.631574,39.142231,152.0
2,10007871,0,2100074550779577850,121.630997,39.142343,DELIVERY,1580529444,20200201,train,0.0,20200201100078710,2,1580528622,2100074550065333539,PICKUP,121.631219,39.141811,121.635154,39.143561,671.0
3,10007871,1,2100074555638285402,121.631208,39.142519,PICKUP,1580532225,20200201,train,1.0,20200201100078711,3,1580532113,2100074554932692192,DELIVERY,121.636904,39.142721,121.636701,39.141801,160.0
4,10007871,1,2100074554118800474,121.631208,39.142519,PICKUP,1580532227,20200201,train,0.0,20200201100078711,4,1580532113,2100074554932692192,DELIVERY,121.636904,39.142721,121.636701,39.141801,160.0


In [14]:
#df_undeleted = df_feature.groupby('tracking_id')
#df_feature0=df_feature
#def repeat1(name, group):
#    if len(group)>1:df_feature.drop(group[group['action_type']=='DELIVERY'].index)
#res = Parallel(n_jobs=3)(delayed(repeat1)(name, group) for name, group in tqdm(df_undeleted))

In [15]:
#feature_grouped=df_feature.groupby('tracking_id')
#df_feature0=df_feature
#deleteList=[]
#for tracking_id in feature_grouped.groups:
#    df_=feature_grouped.get_group(tracking_id)
#    if len(df_)>1:
#        df_feature.drop(df_[df_['action_type']=='DELIVERY'].index)

In [16]:
#不需要重复运行
#df_feature=df_feature0
df=df_feature
df['TA']=df['tracking_id'].astype('str')+df['action_type'].astype('str')
#df=df.sort_values('action_type',ascending=True)
df=df.sort_values('TA',ascending=False)


In [17]:

df.head(10)

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA
221189,122904977,4,2100076535765436020,121.646700,39.126142,PICKUP,0,20200306,test,NaN,202003061229049774,221189,1583502088,2100076535667295820,DELIVERY,121.637154,39.131981,121.628645,39.134531,1016.000000,2100076535765436020PICKUP
221190,122904977,4,2100076535765436020,121.646700,39.126142,DELIVERY,0,20200306,test,NaN,202003061229049774,221190,1583502088,2100076535667295820,DELIVERY,121.637154,39.131981,121.616264,39.137171,2662.000000,2100076535765436020DELIVERY
219026,104989810,4,2100076535342696092,121.789127,39.125969,PICKUP,0,20200306,test,NaN,202003061049898104,219026,1583500619,2100076534535227131,DELIVERY,121.798984,39.122431,121.793299,39.112967,1931.000000,2100076535342696092PICKUP
219029,104989810,4,2100076535342696092,121.789127,39.125969,DELIVERY,0,20200306,test,NaN,202003061049898104,219029,1583500619,2100076534535227131,DELIVERY,121.798984,39.122431,121.803544,39.114371,2125.000000,2100076535342696092DELIVERY
219177,107080018,4,2100076534726690145,121.500613,39.114857,PICKUP,0,20200306,test,NaN,202003061070800184,219177,1583499238,2100076533445690857,DELIVERY,121.507744,39.108871,121.530043,39.115381,2632.000000,2100076534726690145PICKUP
219179,107080018,4,2100076534726690145,121.500613,39.114857,DELIVERY,0,20200306,test,NaN,202003061070800184,219179,1583499238,2100076533445690857,DELIVERY,121.507744,39.108871,121.537954,39.092451,4467.842945,2100076534726690145DELIVERY
220366,118699633,8,2100076534371681354,121.480102,39.184831,PICKUP,0,20200306,test,NaN,202003061186996338,220366,1583498782,2100076532439090219,DELIVERY,121.481614,39.169931,121.474904,39.172700,1019.000000,2100076534371681354PICKUP
220367,118699633,8,2100076534371681354,121.480102,39.184831,DELIVERY,0,20200306,test,NaN,202003061186996338,220367,1583498782,2100076532439090219,DELIVERY,121.481614,39.169931,121.477254,39.164931,1392.000000,2100076534371681354DELIVERY
221187,122904977,4,2100076534237136500,121.646700,39.126142,PICKUP,0,20200306,test,NaN,202003061229049774,221187,1583502088,2100076535667295820,DELIVERY,121.637154,39.131981,121.628645,39.134531,1016.000000,2100076534237136500PICKUP
221188,122904977,4,2100076534237136500,121.646700,39.126142,DELIVERY,0,20200306,test,NaN,202003061229049774,221188,1583502088,2100076535667295820,DELIVERY,121.637154,39.131981,121.618134,39.146901,3701.000000,2100076534237136500DELIVERY


In [18]:
df_feature.shape

(221396, 21)

In [19]:
tracking_id=''
deleteList=[]
for x,row in df.iterrows():
    #print(row['tracking_id'])
    if row['tracking_id']==tracking_id:
        deleteList.append(x)
        #df.drop(x)
    tracking_id=row['tracking_id']

In [20]:
df=df.drop(deleteList)

In [21]:
df=df.sort_values('TA',ascending=True)
df.head(10)

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA
2780,117052073,1,2100074527716017758,121.455805,39.108616,PICKUP,1580528357,20200201,train,1.0,202002011170520731,2780,1580528211,2100074548913145283,PICKUP,121.441945,39.102244,121.439220,39.100178,532.0,2100074527716017758PICKUP
1476,100681730,0,2100074530361804536,121.598811,39.149256,DELIVERY,1580526920,20200201,train,0.0,202002011006817300,1476,1580525615,2100074530361804536,PICKUP,121.601105,39.137024,121.584554,39.116471,4184.0,2100074530361804536DELIVERY
2796,117054417,0,2100074532411639390,121.448985,39.095307,PICKUP,1580525460,20200201,train,1.0,202002011170544170,2796,1580525458,2100074546017437278,PICKUP,121.439220,39.100178,121.439220,39.100178,1.0,2100074532411639390PICKUP
1811,103645298,0,2100074532493362936,121.600431,39.137425,DELIVERY,1580524836,20200201,train,0.0,202002011036452980,1811,1580523872,2100074542979057400,PICKUP,121.601105,39.137024,121.619324,39.129741,2646.0,2100074532493362936DELIVERY
673,10401287,0,2100074533161764858,121.569268,39.104077,PICKUP,1580521211,20200201,train,1.0,20200201104012870,673,1580521209,2100074542338410490,PICKUP,121.575545,39.103083,121.575545,39.103083,1.0,2100074533161764858PICKUP
3610,118916297,0,2100074533397890719,121.977101,39.595172,PICKUP,1580523014,20200201,train,0.0,202002011189162970,3610,1580522123,2100074545051993759,PICKUP,121.960521,39.588041,121.960521,39.588041,1.0,2100074533397890719PICKUP
1608,101915922,1,2100074533420893896,121.690685,39.279492,PICKUP,1580525730,20200201,train,0.0,202002011019159221,1608,1580524864,2100074545447667458,DELIVERY,121.674354,39.272571,121.677846,39.280096,1061.0,2100074533420893896PICKUP
3213,118691289,0,2100074533674616654,121.431624,39.189955,DELIVERY,1580524738,20200201,train,0.0,202002011186912890,3213,1580523464,2100074533674616654,PICKUP,121.419881,39.185003,121.398254,39.169541,3305.0,2100074533674616654DELIVERY
388,10290949,0,2100074533781079886,121.422817,39.184187,DELIVERY,1580522171,20200201,train,0.0,20200201102909490,388,1580521706,2100074539616995150,PICKUP,121.419881,39.185003,121.418574,39.190101,1702.0,2100074533781079886DELIVERY
3139,118662905,0,2100074534060164673,121.543779,39.081136,DELIVERY,1580523911,20200201,train,1.0,202002011186629050,3139,1580523617,2100074543608104054,DELIVERY,121.547704,39.087431,121.547034,39.087641,137.0,2100074534060164673DELIVERY


In [22]:
df.to_pickle('./df')

In [23]:
df_feature=df


# order 表相关特征

In [24]:
df_order.head()

,courier_id,wave_index,tracking_id,weather_grade,pick_lng,pick_lat,deliver_lng,deliver_lat,create_time,confirm_time,assigned_time,promise_deliver_time,estimate_pick_time,aoi_id,shop_id,date
0,100002543,0,2100074536706869066,正常天气,121.647214,39.135549,121.655694,39.123121,1580484067,1580484067,1580519088,1580521500,1580485567,4424963369d15e8900db6cfe732700c3,02615f185581167cc68ba41af3d25ce6,20200201
1,100002543,0,2100074542836254538,正常天气,121.647214,39.135549,121.647324,39.126481,1580517517,1580517517,1580518968,1580521500,1580519017,a986694f4dd7e70ea73e2261889cde81,02615f185581167cc68ba41af3d25ce6,20200201
2,100002543,1,2100074546090674142,正常天气,121.629701,39.139352,121.658144,39.135721,1580523657,1580523658,1580523709,1580526417,1580524558,4df9946bb92f495035be19576fd2d828,374469acda40b1b0b5d34ab4a58cb194,20200201
3,100002543,1,2100074546118723550,正常天气,121.629701,39.139352,121.660584,39.130871,1580523868,1580523869,1580523889,1580526568,1580524589,1e774d8c07c353386658144a16dfd8c1,374469acda40b1b0b5d34ab4a58cb194,20200201
4,100002543,2,2100074572925863604,正常天气,121.642437,39.135199,121.656294,39.122061,1580549262,1580549263,1580549269,1580551902,1580550163,e4bc0664f82df5c8491cbb9f5bd4abb0,9265cec8aa44dceae0072716a3a7e58a,20200201


In [25]:
df_feature = df_feature.merge(
    df_order[['tracking_id', 'weather_grade', 'aoi_id', 'shop_id', 'promise_deliver_time',
              'estimate_pick_time']], how='left')

In [26]:
df_feature.head()

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,weather_grade,aoi_id,shop_id,promise_deliver_time,estimate_pick_time
0,117052073,1,2100074527716017758,121.455805,39.108616,PICKUP,1580528357,20200201,train,1.0,202002011170520731,2780,1580528211,2100074548913145283,PICKUP,121.441945,39.102244,121.439220,39.100178,532.0,2100074527716017758PICKUP,正常天气,1112f462a41e1dc3a774005086d11daa,ca2a062e0790356c72caa2eb909eeb2e,1580530500,1580527009
1,100681730,0,2100074530361804536,121.598811,39.149256,DELIVERY,1580526920,20200201,train,0.0,202002011006817300,1476,1580525615,2100074530361804536,PICKUP,121.601105,39.137024,121.584554,39.116471,4184.0,2100074530361804536DELIVERY,正常天气,e5059162349d42fe9e69450c1b42a33e,8944ec8db309614c49fc787d3ba12f44,1580526900,1580469230
2,117054417,0,2100074532411639390,121.448985,39.095307,PICKUP,1580525460,20200201,train,1.0,202002011170544170,2796,1580525458,2100074546017437278,PICKUP,121.439220,39.100178,121.439220,39.100178,1.0,2100074532411639390PICKUP,正常天气,0ab823ac03cbcae5000924eaf1ea7e12,ca2a062e0790356c72caa2eb909eeb2e,1580526900,1580472595
3,103645298,0,2100074532493362936,121.600431,39.137425,DELIVERY,1580524836,20200201,train,0.0,202002011036452980,1811,1580523872,2100074542979057400,PICKUP,121.601105,39.137024,121.619324,39.129741,2646.0,2100074532493362936DELIVERY,正常天气,5ef6c879fa914a0922fcc5e49f46077b,8944ec8db309614c49fc787d3ba12f44,1580525100,1580521892
4,10401287,0,2100074533161764858,121.569268,39.104077,PICKUP,1580521211,20200201,train,1.0,20200201104012870,673,1580521209,2100074542338410490,PICKUP,121.575545,39.103083,121.575545,39.103083,1.0,2100074533161764858PICKUP,正常天气,6a3dcff2d7fd7a5a3ba955859494bc1c,61be8c5f24588a313c738cc8e68f60a5,1580523300,1580516006


# courier 表相关特征

In [27]:
df_feature = df_feature.merge(df_courier, how='left')

In [28]:
df_feature.to_pickle('./temp/part1_feature.plk')

In [29]:
df_feature.shape

(150060, 29)

In [30]:
df_feature.head(20)

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,weather_grade,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,level,speed,max_load
0,117052073,1,2100074527716017758,121.455805,39.108616,PICKUP,1580528357,20200201,train,1.0,202002011170520731,2780,1580528211,2100074548913145283,PICKUP,121.441945,39.102244,121.439220,39.100178,532.0,2100074527716017758PICKUP,正常天气,1112f462a41e1dc3a774005086d11daa,ca2a062e0790356c72caa2eb909eeb2e,1580530500,1580527009,3,4.643877,11
1,100681730,0,2100074530361804536,121.598811,39.149256,DELIVERY,1580526920,20200201,train,0.0,202002011006817300,1476,1580525615,2100074530361804536,PICKUP,121.601105,39.137024,121.584554,39.116471,4184.0,2100074530361804536DELIVERY,正常天气,e5059162349d42fe9e69450c1b42a33e,8944ec8db309614c49fc787d3ba12f44,1580526900,1580469230,2,4.489853,8
2,117054417,0,2100074532411639390,121.448985,39.095307,PICKUP,1580525460,20200201,train,1.0,202002011170544170,2796,1580525458,2100074546017437278,PICKUP,121.439220,39.100178,121.439220,39.100178,1.0,2100074532411639390PICKUP,正常天气,0ab823ac03cbcae5000924eaf1ea7e12,ca2a062e0790356c72caa2eb909eeb2e,1580526900,1580472595,3,5.228923,11
3,103645298,0,2100074532493362936,121.600431,39.137425,DELIVERY,1580524836,20200201,train,0.0,202002011036452980,1811,1580523872,2100074542979057400,PICKUP,121.601105,39.137024,121.619324,39.129741,2646.0,2100074532493362936DELIVERY,正常天气,5ef6c879fa914a0922fcc5e49f46077b,8944ec8db309614c49fc787d3ba12f44,1580525100,1580521892,3,4.371049,9
4,10401287,0,2100074533161764858,121.569268,39.104077,PICKUP,1580521211,20200201,train,1.0,20200201104012870,673,1580521209,2100074542338410490,PICKUP,121.575545,39.103083,121.575545,39.103083,1.0,2100074533161764858PICKUP,正常天气,6a3dcff2d7fd7a5a3ba955859494bc1c,61be8c5f24588a313c738cc8e68f60a5,1580523300,1580516006,3,4.768473,7
5,118916297,0,2100074533397890719,121.977101,39.595172,PICKUP,1580523014,20200201,train,0.0,202002011189162970,3610,1580522123,2100074545051993759,PICKUP,121.960521,39.588041,121.960521,39.588041,1.0,2100074533397890719PICKUP,正常天气,090290b5354073fa64a3266d81c9d09f,d6265126edc531b3cb13a00ed9ccd6bd,1580525100,1580475782,3,4.685788,9
6,101915922,1,2100074533420893896,121.690685,39.279492,PICKUP,1580525730,20200201,train,0.0,202002011019159221,1608,1580524864,2100074545447667458,DELIVERY,121.674354,39.272571,121.677846,39.280096,1061.0,2100074533420893896PICKUP,正常天气,dd32ff9b2aa697b0792e4e27eaa9d904,ac5cd13db63c15230b57b3d1fffbdbf2,1580526900,1580476426,3,4.736526,8
7,118691289,0,2100074533674616654,121.431624,39.189955,DELIVERY,1580524738,20200201,train,0.0,202002011186912890,3213,1580523464,2100074533674616654,PICKUP,121.419881,39.185003,121.398254,39.169541,3305.0,2100074533674616654DELIVERY,正常天气,f25cbfe835006fd049a069c952c5ee8a,52609cbee4c8444396b944529cd0856d,1580525100,1580474743,3,4.725414,10
8,10290949,0,2100074533781079886,121.422817,39.184187,DELIVERY,1580522171,20200201,train,0.0,20200201102909490,388,1580521706,2100074539616995150,PICKUP,121.419881,39.185003,121.418574,39.190101,1702.0,2100074533781079886DELIVERY,正常天气,f2760ca7214e0081158a43d0d20c8245,52609cbee4c8444396b944529cd0856d,1580523300,1580520790,0,4.699998,9
9,118662905,0,2100074534060164673,121.543779,39.081136,DELIVERY,1580523911,20200201,train,1.0,202002011186629050,3139,1580523617,2100074543608104054,DELIVERY,121.547704,39.087431,121.547034,39.087641,137.0,2100074534060164673DELIVERY,正常天气,172cd36588e2214177d25a13215e9d51,ff4dc09a24956989e98b205b8af41697,1580525100,1580476524,3,4.759711,10
